In [ ]:
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy
import tqdm
import math
import torch
import gpytorch
from matplotlib import pyplot as plt
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

class GPModel(ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(GPModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

n_train   = 600
n_test = 200
dim = 2

train_x = torch.rand(n_train,dim)
test_x = torch.rand(n_test,dim)

# f(x) = sin(2pi(x**2+y**2)), df/dx = cos(2pi(x**2+y**2))4pi*x, df/dy = cos(2pi(x**2+y**2))4pi*y
train_y = torch.sin(2*np.pi*(train_x[:,0]**2+train_x[:,1]**2))
test_y = torch.sin(2*np.pi*(test_x[:,0]**2+test_x[:,1]**2))
# train_y = torch.sin(2*np.pi*train_x[:,0])

num_inducing = 20
batch_size = int(n_train/2)

inducing_points = train_x[:num_inducing, :]
# inducing_points = torch.rand(num_inducing,dim)
model = GPModel(inducing_points=inducing_points)
likelihood = gpytorch.likelihoods.GaussianLikelihood()

if torch.cuda.is_available():
    model = model.cuda()
    likelihood = likelihood.cuda()

num_epochs = 1000


model.train()
likelihood.train()

optimizer = torch.optim.Adam([
    {'params': model.parameters()},
    {'params': likelihood.parameters()},
], lr=0.01)

# Our loss object. We're using the VariationalELBO
mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_y.size(0))



train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(train_x, train_y)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

epochs_iter = tqdm.tqdm(range(num_epochs), desc="Epoch")
for i in epochs_iter:
    # Within each iteration, we will go over each minibatch of data
    minibatch_iter = tqdm.tqdm(train_loader, desc="Minibatch", leave=False)
    for x_batch, y_batch in minibatch_iter:
        optimizer.zero_grad()
        output = model(x_batch)
        loss = -mll(output, y_batch)
        epochs_iter.set_postfix(loss=loss.item())
        loss.backward()
        optimizer.step()

model.eval()
likelihood.eval()

for param in model.parameters():
  if param.requires_grad:
    print(param.data)

means = torch.tensor([0.])
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        preds = model(x_batch)
        means = torch.cat([means, preds.mean.cpu()])
print("MEANS")
means = means[1:]

print("")
print("Testing Mean Squared Error: ", np.var(means.detach().numpy() - train_y.detach().numpy()))

from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(train_x[:,0],train_x[:,1],train_y, color='k')
ax.scatter(train_x[:,0],train_x[:,1],means, color='b')
plt.title("f(x,y) variational fit; actual curve is black, variational is blue")
plt.show()

Epoch:   0%|                                                                       | 0/1000 [00:00<?, ?it/s, loss=1.56]
                                                                                                                       
Epoch:   0%|                                                                       | 0/1000 [00:00<?, ?it/s, loss=1.49]
                                                                                                                       
Epoch:   0%|▏                                                              | 3/1000 [00:00<00:45, 21.74it/s, loss=1.41]
                                                                                                                       
Epoch:   0%|▏                                                              | 3/1000 [00:00<00:45, 21.74it/s, loss=1.35]
                                                                                                                       
Epoch:   1%|▍                           

Epoch:   3%|██                                                           | 33/1000 [00:01<00:47, 20.56it/s, loss=0.967]
                                                                                                                       
Epoch:   3%|██                                                           | 33/1000 [00:01<00:47, 20.56it/s, loss=0.983]
                                                                                                                       
Epoch:   4%|██▏                                                          | 36/1000 [00:01<00:43, 22.10it/s, loss=0.962]
                                                                                                                       
Epoch:   4%|██▏                                                          | 36/1000 [00:01<00:43, 22.10it/s, loss=0.939]
                                                                                                                       
Epoch:   4%|██▍                         

Epoch:   7%|████                                                          | 66/1000 [00:03<00:40, 23.26it/s, loss=0.65]
                                                                                                                       
Epoch:   7%|████                                                         | 66/1000 [00:03<00:40, 23.26it/s, loss=0.643]
                                                                                                                       
Epoch:   7%|████▏                                                        | 69/1000 [00:03<00:39, 23.59it/s, loss=0.584]
                                                                                                                       
Epoch:   7%|████▏                                                        | 69/1000 [00:03<00:39, 23.59it/s, loss=0.584]
                                                                                                                       
Epoch:   7%|████▍                       

Epoch:  10%|██████                                                       | 99/1000 [00:04<00:40, 22.07it/s, loss=0.213]
                                                                                                                       
Epoch:  10%|██████                                                       | 99/1000 [00:04<00:40, 22.07it/s, loss=0.202]
                                                                                                                       
Epoch:  10%|██████                                                      | 102/1000 [00:04<00:39, 22.98it/s, loss=0.175]
                                                                                                                       
Epoch:  10%|██████▏                                                      | 102/1000 [00:04<00:39, 22.98it/s, loss=0.17]
                                                                                                                       
Epoch:  10%|██████▎                     

Epoch:  13%|███████▊                                                   | 132/1000 [00:05<00:41, 20.69it/s, loss=-.0542]
                                                                                                                       
Epoch:  13%|███████▊                                                   | 132/1000 [00:06<00:41, 20.69it/s, loss=-.0562]
                                                                                                                       
Epoch:  14%|███████▉                                                   | 135/1000 [00:06<00:41, 20.73it/s, loss=-.0894]
                                                                                                                       
Epoch:  14%|████████                                                    | 135/1000 [00:06<00:41, 20.73it/s, loss=-.114]
                                                                                                                       
Epoch:  14%|████████▎                   

Epoch:  16%|█████████▉                                                  | 165/1000 [00:07<00:38, 21.43it/s, loss=-.211]
                                                                                                                       
Epoch:  16%|█████████▉                                                  | 165/1000 [00:07<00:38, 21.43it/s, loss=-.182]
                                                                                                                       
Epoch:  17%|██████████▏                                                  | 168/1000 [00:07<00:37, 21.90it/s, loss=-.18]
                                                                                                                       
Epoch:  17%|██████████                                                  | 168/1000 [00:07<00:37, 21.90it/s, loss=-.213]
                                                                                                                       
Epoch:  17%|██████████▎                 

Epoch:  20%|███████████▉                                                | 198/1000 [00:08<00:35, 22.59it/s, loss=-.271]
                                                                                                                       
Epoch:  20%|███████████▉                                                | 198/1000 [00:08<00:35, 22.59it/s, loss=-.207]
                                                                                                                       
Epoch:  20%|████████████                                                | 201/1000 [00:09<00:35, 22.23it/s, loss=-.224]
                                                                                                                       
Epoch:  20%|████████████                                                | 201/1000 [00:09<00:35, 22.23it/s, loss=-.282]
                                                                                                                       
Epoch:  20%|████████████▏               